In [30]:
import json
# EXPERIMENTAL
# Work in progress (do not use)

# find TC genesis from models based on work similar to FSU's methodology

# only focused on 1. so far:
# 1. https://journals.ametsoc.org/view/journals/wefo/28/6/waf-d-13-00008_1.xml
# 2. https://journals.ametsoc.org/view/journals/wefo/31/3/waf-d-15-0157_1.xml?tab_body=fulltext-display
# 3. https://journals.ametsoc.org/view/journals/wefo/32/1/waf-d-16-0072_1.xml

# thresholds for disturbances (reversed from text output from FSU)
disturbance_thresholds_path = 'disturbance_thresholds.json'

# Test the function with your GRIB file
# NAVGEM 96h
# uses regular_ll

#grib_file = "navgem_2023101906f096.grib2"
# cgi combined all relevant?
# the new was with cgi but it gets TOO much information (it gets 11 parameters rather than 9: u/v wind for 200 we don't need)
##### grib_file = "/home/db/metview/JRPdata/gfs/gfs.t00z.pgrb2.0p25.f000_new"
#   use the index file
#grib_file = "/home/db/metview/JRPdata/navgem2/US058GMET-GR1mdl.0018_0056_09600F0OF2023101906_0100_008500-000000geop_ht"
#grib_file = "/home/db/metview/JRPdata/ukmet/isbl_geopotential-height_85000.0_2023102112_12.grib"

model_name = 'NAV'

disturbance_criteria_names_map = {
    'WS': 'vmax925',
    'THKN': 'gp250_850_thickness',
    'RV': 'rv850max'
}

with open(disturbance_thresholds_path, 'r') as f:
    disturbance_thresholds = json.loads(f.read())

In [22]:
list_available_parameters(grib_file)

1:Geopotential Height:gpm (instant):regular_ll:isobaricInhPa:level 20000 Pa:fcst time 0 hrs:from 202310210000
2:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 20000 Pa:fcst time 0 hrs:from 202310210000
3:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 20000 Pa:fcst time 0 hrs:from 202310210000
4:Geopotential Height:gpm (instant):regular_ll:isobaricInhPa:level 85000 Pa:fcst time 0 hrs:from 202310210000
5:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 85000 Pa:fcst time 0 hrs:from 202310210000
6:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 85000 Pa:fcst time 0 hrs:from 202310210000
7:Geopotential Height:gpm (instant):regular_ll:isobaricInhPa:level 92500 Pa:fcst time 0 hrs:from 202310210000
8:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 92500 Pa:fcst time 0 hrs:from 202310210000
9:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 92500 Pa:fcst time 0 hrs:from

In [15]:
import pygrib
import re
import math
from metpy.units import units
import metpy.calc as mpcalc
import numpy as np
import pyproj

# may need to modify lzma.py in python folder to get this to work (pip install backports-lzma)
try:
    import lzma
except ImportError:
    import backports.lzma as lzma

In [62]:
# todo: the calculations using neighborhoods don't wrap around at lons (and lats) and they should
#  for instance, the unsigned lats array starts at 0 degrees and ends at 360

def list_available_parameters(grib_file):
    try:
        # Open the GRIB file
        grbs = pygrib.open(grib_file)

        # Initialize a list to store parameter information
        parameter_info = []

        # Iterate through the GRIB messages and extract parameter information
        for grb in grbs:
            parameter_name = grb.name
            parameter_unit = grb.units
            level_type = grb.levelType
            level = grb.level
            print(grb)
            parameter_info.append({
                "Parameter Name": parameter_name,
                "Unit": parameter_unit,
                "Level Type": level_type,
                "Level": level
            })

        # Close the GRIB file
        grbs.close()

        # Print the information for parameters
        for info in parameter_info:
            print("Parameter Name:", info["Parameter Name"])
            print("Unit:", info["Unit"])
            print("Level Type:", info["Level Type"])
            print("Level:", info["Level"])
            print("\n")

    except Exception as e:
        print(f"Error: {e}")

def load_and_extract_grib_data(grib_file):
    try:
        # Open the GRIB file
        grbs = pygrib.open(grib_file)

        # Initialize variables for relevant parameters
        mslp = None
        u_wind_925 = None
        v_wind_925 = None
        u_wind_850 = None
        v_wind_850 = None
        geopotential_250 = None
        geopotential_850 = None
        
        mslp_units = None
        wind_925_units = None
        
        mslp_lats = None
        mslp_lons = None
        u_wind_925_lats = None
        u_wind_925_lons = None
        v_wind_925_lats = None
        v_wind_925_lons = None
        u_wind_850_lats = None
        u_wind_850_lons = None
        v_wind_850_lats = None
        v_wind_850_lons = None
        geopotential_250_lats = None
        geopotential_250_lons = None
        geopotential_850_lats = None
        geopotential_850_lons = None
        
        grid_resolution = {}
        
        # Extract relevant parameters (modify the parameter names and levels accordingly)
        for grb in grbs:
            if grb.name == 'Pressure reduced to MSL' and grb.level == 0:
                mslp = grb.values
                mslp_units = grb.units
                mslp_lats, mslp_lons = grb.latlons()
                grid_resolution['mslp'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'U component of wind' and grb.level == 925:
                u_wind_925 = grb.values
                wind_925_units = grb.units
                u_wind_925_lats, u_wind_925_lons = grb.latlons()
                grid_resolution['uwind925'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'V component of wind' and grb.level == 925:
                v_wind_925 = grb.values
                v_wind_925_lats, v_wind_925_lons = grb.latlons()
                grid_resolution['vwind925'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'U component of wind' and grb.level == 850:
                u_wind_850 = grb.values
                wind_850_units = grb.units
                u_wind_850_lats, u_wind_850_lons = grb.latlons()
                grid_resolution['uwind850'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'V component of wind' and grb.level == 850:
                v_wind_850 = grb.values
                v_wind_850_lats, v_wind_850_lons = grb.latlons()
                grid_resolution['vwind850'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'Geopotential Height' and grb.level == 250:
                geopotential_250 = grb.values
                geopotential_250_lats, geopotential_250_lons = grb.latlons()
                grid_resolution['gh250'] = grb['iDirectionIncrementInDegrees']
            elif grb.name == 'Geopotential Height' and grb.level == 850:
                geopotential_850 = grb.values
                geopotential_850_lats, geopotential_850_lons = grb.latlons()
                grid_resolution['gh850'] = grb['iDirectionIncrementInDegrees']

        # Close the GRIB file
        grbs.close()


        # Check if units are not in hPa and convert if necessary
        if mslp_units != "hPa":
            print(f"Converting MSLP units from {mslp_units} to hPa")
            if mslp_units == "Pa":
                # Convert from Pa to hPa
                mslp *= 0.01
                mslp_units = "hPa"
            else:
                print("Warning: Units of MSLP are not in Pa or hPa. Please verify the units for accurate results.")

        # Check if units are not in m/s for 925 hPa wind components and convert if necessary
        if not (re.search(r"(m/s|m s\*\*-1)", wind_925_units)):
            print(f"Converting 925 hPa wind components units from {wind_925_units} to m/s")
            if re.search(r"knots|knot", wind_925_units, re.I):
                # Convert from knots to m/s (1 knot ≈ 0.514444 m/s)
                u_wind_925 *= 0.514444
                v_wind_925 *= 0.514444
                wind_925_units = "m/s"
            else:
                print("Warning: Units of 925 hPa wind components are not in knots, m/s, or m s**-1. Please verify the units for accurate results.")
                
        # Check if units are not in m/s for 925 hPa wind components and convert if necessary
        if not (re.search(r"(m/s|m s\*\*-1)", wind_850_units)):
            print(f"Converting 850 hPa wind components units from {wind_850_units} to m/s")
            if re.search(r"knots|knot", wind_925_units, re.I):
                # Convert from knots to m/s (1 knot ≈ 0.514444 m/s)
                u_wind_850 *= 0.514444
                v_wind_850 *= 0.514444
                wind_850_units = "m/s"
            else:
                print("Warning: Units of 925 hPa wind components are not in knots, m/s, or m s**-1. Please verify the units for accurate results.")
        
        return grid_resolution, mslp_units, mslp, mslp_lats, mslp_lons, wind_925_units, u_wind_925, u_wind_925_lats, u_wind_925_lons, v_wind_925, v_wind_925_lats, v_wind_925_lons, wind_850_units, u_wind_850, u_wind_850_lats, u_wind_850_lons, v_wind_850, v_wind_850_lats, v_wind_850_lons, geopotential_250, geopotential_250_lats, geopotential_250_lons, geopotential_850, geopotential_850_lats, geopotential_850_lons
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None, None, None

def convert_to_signed_lon(lon):
    # Convert longitudes from 0-360 range to -180 to +180 range
    return (lon + 180) % 360 - 180


def calculate_neighborhood_size(degree_radius, grid_resolution):
    # Calculate the radius in grid points
    radius_in_grid_points = int(degree_radius / grid_resolution)

    # Calculate the neighborhood size
    neighborhood_size = 2 * radius_in_grid_points + 1  # Assuming a square neighborhood

    return neighborhood_size

# print candidates
def print_candidates(mslp_minima_list, mslp_lats, mslp_lons, meet_all_disturbance_thresholds = False):
    n = 0
    for mslp_minima_data in mslp_minima_list:
        if meet_all_disturbance_thresholds:
            if not mslp_minima_data['criteria']['all']:
                continue
        n += 1
        mslp_value = mslp_minima_data["mslp_value"]
        x = mslp_minima_data["x_value"]
        y = mslp_minima_data["y_value"]
        
        lat = mslp_lats[x, y]
        lon = mslp_lons[x, y]
        signed_lon = convert_to_signed_lon(lon)
        
        formatted_mslp = f"{mslp_value:.1f}".rjust(6, ' ')
        rv_str = ""
        if 'rv850max'in mslp_minima_data:
            rv_str = ", 850 RV MAX (*10^-5 1/s): "
            rv_value = mslp_minima_data['rv850max'] * np.power(10.0,5)
            formatted_rv = f"{rv_value:2.2f}".rjust(5, ' ')
            rv_str += formatted_rv
        
        thickness_str = ""
        if 'gp250_850_thickness' in mslp_minima_data:
            thickness_str = ", 250-850 hPa Thickness (m): "
            thickness_value = mslp_minima_data['gp250_850_thickness']
            formatted_thickness = f"{thickness_value:2.2f}".rjust(6, ' ')
            thickness_str += formatted_thickness
        
        vmax_str = ""
        if 'vmax925' in mslp_minima_data:
            vmax_str = ", 925 hPa WS MAX (m/s): "
            vmax_value = mslp_minima_data['vmax925']
            formatted_vmax = f"{vmax_value:3.2f}".rjust(6, ' ')
            vmax_str += formatted_vmax
        
        print(f"#{n: >2}, Latitude (deg): {lat: >6}, Longitude (deg): {signed_lon: >6}, MSLP (hPa): {formatted_mslp}{rv_str}{thickness_str}{vmax_str}")

# this function is for checking find_mslp_minima_with_closed_isobars
def find_mslp_minima(mslp_data, minima_neighborhood_size=3):
    try:
        # Lists to store MSLP minima, latitudes, and longitudes
        mslp_minima_list = []

        # Loop through each grid point
        for x in range(minima_neighborhood_size, mslp_data.shape[0] - minima_neighborhood_size):
            for y in range(minima_neighborhood_size, mslp_data.shape[1] - minima_neighborhood_size):
                mslp_value = mslp_data[x, y]  # MSLP value at the current point
                neighborhood = mslp_data[x - minima_neighborhood_size:x + minima_neighborhood_size + 1,
                                         y - minima_neighborhood_size:y + minima_neighborhood_size + 1]

                # Check if the MSLP value is the minimum within the neighborhood
                if mslp_value == neighborhood.min():
                    mslp_minima_data = {
                        "mslp_value": mslp_value,
                        "x_value": x,
                        "y_value": y
                    }
                    mslp_minima_list.append(mslp_minima_data)


        return mslp_minima_list
    except Exception as e:
        print(f"Error: {e}")
        return [], [], []

def find_mslp_minima_with_closed_isobars(mslp_data, grid_resolution, minima_neighborhood_size=3, isobar_threshold=2.0, isobar_search_radius_degrees = 5):
    isobar_neighborhood_size = calculate_neighborhood_size(isobar_search_radius_degrees, grid_resolution)
    
    def dfs(x, y):
        # Depth-first search to find a closed path
        nonlocal path_found
        visited[x][y] = True

        for dx in range(-1, 2):
            for dy in range(-1, 2):
                nx, ny = x + dx, y + dy

                if 0 <= nx < isobar_neighborhood_size * 2 + 1 and 0 <= ny < isobar_neighborhood_size * 2 + 1 and not visited[nx][ny]:
                    if neighborhood_modified[nx][ny] >= 0:
                        if nx == 0 or ny == 0 or nx == isobar_neighborhood_size * 2 or ny == isobar_neighborhood_size * 2:
                            path_found = True
                            return
                        dfs(nx, ny)

    try:
        # List to store MSLP minima as dictionaries
        mslp_minima_list = []

        # Create a list of candidates for MSLP minima
        candidates = []

        # Loop through each grid point
        for x in range(minima_neighborhood_size, mslp_data.shape[0] - minima_neighborhood_size):
            for y in range(minima_neighborhood_size, mslp_data.shape[1] - minima_neighborhood_size):
                mslp_value = mslp_data[x, y]  # MSLP value at the current point
                neighborhood = mslp_data[x - minima_neighborhood_size:x + minima_neighborhood_size + 1,
                                         y - minima_neighborhood_size:y + minima_neighborhood_size + 1]

                # Check if the MSLP value is the minimum within the neighborhood
                if mslp_value == neighborhood.min():
                    candidates.append((x, y, mslp_value))

        # Loop through the candidates to find isobars
        for x, y, minima_value in candidates:
            # Create a modified neighborhood for isobar calculation
            x_min = max(0, x - isobar_neighborhood_size)
            x_max = min(mslp_data.shape[0], x + isobar_neighborhood_size + 1)
            y_min = max(0, y - isobar_neighborhood_size)
            y_max = min(mslp_data.shape[1], y + isobar_neighborhood_size + 1)

            neighborhood_modified = mslp_data[x_min:x_max, y_min:y_max] - minima_value - isobar_threshold

            # Initialize visited array for DFS
            visited = np.zeros_like(neighborhood_modified, dtype=bool)

            # Flag to track if a closed path is found
            path_found = False

            # Start DFS from the center of the neighborhood
            dfs(x - x_min, y - y_min)

            if path_found:
                # Store MSLP minima data as a dictionary
                mslp_minima_data = {
                    "mslp_value": minima_value,
                    "x_value": x,
                    "y_value": y
                }
                mslp_minima_list.append(mslp_minima_data)

        return mslp_minima_list

    except Exception as e:
        print(f"Error: {e}")
        return []

# calculate vorticity (accounting for projection distortion) using metpy (uses finite differences method)
# returns vorticity using an ellipsoid based on WGS84
def calculate_vorticity(u_wind_850, v_wind_850, mslp_lons, mslp_lats):
    # Calculate grid spacing distances (these will be the dx and dy values used for calculating vorticity)
    ellipsoid = pyproj.Geod(ellps="WGS84")
    mslp_dx, mslp_dy = mpcalc.lat_lon_grid_deltas(mslp_lons * units.degrees, mslp_lats * units.degrees, geod=ellipsoid)

    # Convert wind components to units
    u_wind_850_with_units = units.Quantity(u_wind_850, 'm/s')
    v_wind_850_with_units = units.Quantity(v_wind_850, 'm/s')

    # Calculate relative vorticity (use WGS84)
    vort_850 = np.array(mpcalc.vorticity(u_wind_850_with_units, v_wind_850_with_units, dx=mslp_dx, dy=mslp_dy))

    return vort_850

# Function to find RV maximums in neighborhoods for a list of candidates
def find_rv_maximums_in_neighborhoods(mslp_minima_list, rv, grid_resolution, relative_vorticity_radius_degrees = 2):
    rv_neighborhood_size = calculate_neighborhood_size(relative_vorticity_radius_degrees, grid_resolution)
    updated_mslp_minima_list = []

    for candidate in mslp_minima_list:
        x, y, _ = candidate["x_value"], candidate["y_value"], candidate["mslp_value"]

        # Extract the neighborhood for the current candidate
        x_min = max(0, x - rv_neighborhood_size)
        x_max = min(rv.shape[0], x + rv_neighborhood_size + 1)
        y_min = max(0, y - rv_neighborhood_size)
        y_max = min(rv.shape[1], y + rv_neighborhood_size + 1)
        neighborhood = rv[x_min:x_max, y_min:y_max]

        # Find the maximum RV value within the neighborhood
        rv_max = neighborhood.max()

        # Update a copy of the candidate's dictionary with rv maximum value
        updated_candidate = candidate.copy()
        updated_candidate["rv850max"] = rv_max

        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list
    
# calculate relative max thickness for 250hPa - 850hPa for a list of candidates
def find_gp_250_850_max_thickness(mslp_minima_list, geopotential_250, geopotential_850, grid_resolution, degrees_radius=2):
    # Calculate the neighborhood size based on degrees_radius and grid_resolution
    neighborhood_size = calculate_neighborhood_size(degrees_radius, grid_resolution)

    updated_mslp_minima_list = []
    
    # Get the shape of the geopotential arrays
    array_shape = geopotential_250.shape

    # Iterate over the list of candidates
    for candidate in mslp_minima_list:
        # Extract the x and y indices of the candidate
        x_index, y_index = candidate['x_value'], candidate['y_value']

        # Calculate the neighborhood bounds
        x_start = max(0, x_index - neighborhood_size)
        x_end = min(array_shape[0] - 1, x_index + neighborhood_size)
        y_start = max(0, y_index - neighborhood_size)
        y_end = min(array_shape[1] - 1, y_index + neighborhood_size)

        # Extract the neighborhoods for 250 hPa and 850 hPa
        neighborhood_250 = geopotential_250[x_start:x_end + 1, y_start:y_end + 1]
        neighborhood_850 = geopotential_850[x_start:x_end + 1, y_start:y_end + 1]

        # Calculate the 250–850-hPa thickness for each cell in the neighborhood
        thickness = neighborhood_250 - neighborhood_850

        # Find the maximum thickness value in the neighborhood
        max_thickness = thickness.max()

        # Update a copy of the candidate's dictionary with the maximum thickness value
        updated_candidate = candidate.copy()
        updated_candidate['gp250_850_thickness'] = max_thickness
        
        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

# find max wind at 925 hPa for a list of candidates
def find_max_wind_925(mslp_minima_list, u_wind_925, v_wind_925, grid_resolution, degrees_radius=5):
    # Calculate the neighborhood size based on radius_degrees and grid_resolution for wind
    neighborhood_size = calculate_neighborhood_size(degrees_radius, grid_resolution)

    updated_mslp_minima_list = []
    
    # Get the shape of the wind arrays
    array_shape = u_wind_925.shape

    # Iterate over the list of candidates
    for candidate in mslp_minima_list:
        # Extract the x and y indices of the candidate
        x_index = candidate['x_value']
        y_index = candidate['y_value']

        # Calculate the neighborhood bounds
        x_start = max(0, x_index - neighborhood_size)
        x_end = min(array_shape[0] - 1, x_index + neighborhood_size)
        y_start = max(0, y_index - neighborhood_size)
        y_end = min(array_shape[1] - 1, y_index + neighborhood_size)

        # Extract the neighborhoods for u-wind and v-wind at 925 hPa
        neighborhood_u_wind = u_wind_925[x_start:x_end + 1, y_start:y_end + 1]
        neighborhood_v_wind = v_wind_925[x_start:x_end + 1, y_start:y_end + 1]

        # Calculate wind speed in the neighborhood
        wind_speed = np.sqrt(neighborhood_u_wind ** 2 + neighborhood_v_wind ** 2)

        # Find the maximum wind speed value in the neighborhood
        max_wind_speed = wind_speed.max()

        # Update a copy of the candidate's dictionary with the maximum wind speed value
        updated_candidate = candidate.copy()
        updated_candidate['vmax925'] = max_wind_speed
        
        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

In [4]:
# Test the function
grib_file = "navgem_2023101906f096.grib2"
grid_resolution, mslp_units, mslp, mslp_lats, mslp_lons, wind_925_units, u_wind_925, u_wind_925_lats, u_wind_925_lons, v_wind_925, v_wind_925_lats, v_wind_925_lons, wind_850_units, u_wind_850, u_wind_850_lats, u_wind_850_lons, v_wind_850, v_wind_850_lats, v_wind_850_lons, geopotential_250, geopotential_250_lats, geopotential_250_lons, geopotential_850, geopotential_850_lats, geopotential_850_lons = load_and_extract_grib_data(grib_file)

# Print the extracted data (for testing purposes)
print("MSLP Data:", mslp)
print("U Wind at 925 hPa:", u_wind_925)
print("V Wind at 925 hPa:", v_wind_925)
print("U Wind at 850 hPa:", u_wind_850)
print("V Wind at 850 hPa:", v_wind_850)
print("Geopotential Height at 250 hPa:", geopotential_250)
print("Geopotential Height at 850 hPa:", geopotential_850)

Converting MSLP units from Pa to hPa
MSLP Data: [[1020.74 1020.74 1020.74 ... 1020.74 1020.74 1020.74]
 [1019.25 1019.25 1019.28 ... 1019.23 1019.22 1019.22]
 [1017.28 1017.28 1017.32 ... 1017.26 1017.22 1017.23]
 ...
 [1021.68 1021.69 1021.7  ... 1021.67 1021.67 1021.68]
 [1022.02 1022.02 1022.02 ... 1022.01 1022.01 1022.02]
 [1022.38 1022.38 1022.38 ... 1022.38 1022.38 1022.38]]
U Wind at 925 hPa: [[ 2.39 -0.54 -0.54 ... -1.07 -1.02 -0.98]
 [-2.   -1.97 -1.93 ... -2.14 -2.1  -2.06]
 [-1.02 -1.   -0.98 ... -1.11 -1.08 -1.05]
 ...
 [-4.45 -4.41 -4.37 ... -4.58 -4.54 -4.5 ]
 [-4.58 -4.55 -4.51 ... -4.69 -4.65 -4.62]
 [ 6.43 -5.12 -5.12 ... -5.5  -5.46 -5.42]]
V Wind at 925 hPa: [[-6.28 -5.28 -5.29 ... -5.25 -5.26 -5.27]
 [-6.1  -6.12 -6.15 ... -6.04 -6.06 -6.08]
 [-5.78 -5.79 -5.81 ... -5.74 -5.75 -5.77]
 ...
 [ 4.21  4.25  4.29 ...  4.08  4.12  4.16]
 [ 4.17  4.21  4.25 ...  4.06  4.1   4.13]
 [ 4.97  4.29  4.34 ...  4.1   4.14  4.14]]
U Wind at 850 hPa: [[-2.08 -2.21 -2.11 ... -1.75 -

In [16]:
list_available_parameters(grib_file)

Error: [Errno could not open %s] gfs.t06z.pgrb2.0p25_200_850_925.f000


In [6]:
# Call the function with your data
mslp_minima_list = find_mslp_minima(mslp)

In [7]:
# list of all candidates by minima (without a closed isobar at an interval of 2hPa)
print_candidates(mslp_minima_list, mslp_lats, mslp_lons)

# 1, Latitude (deg):  -84.5, Longitude (deg): -168.5, MSLP (hPa):  992.4
# 2, Latitude (deg):  -84.0, Longitude (deg):  174.5, MSLP (hPa):  992.5
# 3, Latitude (deg):  -84.0, Longitude (deg): -178.0, MSLP (hPa):  993.3
# 4, Latitude (deg):  -84.0, Longitude (deg): -173.0, MSLP (hPa):  992.7
# 5, Latitude (deg):  -83.0, Longitude (deg):  166.0, MSLP (hPa):  993.0
# 6, Latitude (deg):  -83.0, Longitude (deg):  166.5, MSLP (hPa):  993.0
# 7, Latitude (deg):  -82.5, Longitude (deg):  153.5, MSLP (hPa):  993.6
# 8, Latitude (deg):  -82.5, Longitude (deg):  161.0, MSLP (hPa):  993.3
# 9, Latitude (deg):  -81.0, Longitude (deg):   64.5, MSLP (hPa): 1027.2
#10, Latitude (deg):  -81.0, Longitude (deg):   65.0, MSLP (hPa): 1027.2
#11, Latitude (deg):  -80.5, Longitude (deg):   47.5, MSLP (hPa): 1024.7
#12, Latitude (deg):  -80.5, Longitude (deg):   79.5, MSLP (hPa): 1024.7
#13, Latitude (deg):  -79.5, Longitude (deg):   93.0, MSLP (hPa): 1023.3
#14, Latitude (deg):  -79.5, Longitude (deg):   99.

In [8]:
# this will be the candidates restricted by MSLP closed isobars only
mslp_minima_list_with_closed_isobars = find_mslp_minima_with_closed_isobars(mslp, grid_resolution['mslp'])

# calculate relative vorticity for the entire data set first
# this may issue warnings for divide by 0 for vorticity calculations
rv_850 = calculate_vorticity(u_wind_850, v_wind_850, mslp_lons, mslp_lats)
# calculate relative vorticity maximum for each candidate
mslp_minima_list_with_closed_isobars = find_rv_maximums_in_neighborhoods(mslp_minima_list_with_closed_isobars, rv_850, grid_resolution['gh250'])

# Find the maximum relative thickness (250 - 850 hPa) for each candidate
mslp_minima_list_with_closed_isobars = find_gp_250_850_max_thickness(mslp_minima_list_with_closed_isobars, geopotential_250, geopotential_850, grid_resolution['gh250'])

# find the maximum 925 hPa wind speed
mslp_minima_list_with_closed_isobars = find_max_wind_925(mslp_minima_list_with_closed_isobars, u_wind_925, v_wind_925, grid_resolution['uwind925'])

/home/db/.local/lib/python3.9/site-packages/pint/facets/plain/quantity.py:986: RuntimeWarning: invalid value encountered in divide
  magnitude = magnitude_op(new_self._magnitude, other._magnitude)


In [9]:
print_candidates(mslp_minima_list_with_closed_isobars, mslp_lats, mslp_lons)

# 1, Latitude (deg):  -12.5, Longitude (deg):  167.0, MSLP (hPa):  982.3, 850 RV MAX (*10^-5 1/s):  4.05, 250-850 hPa Thickness (m): 9664.80, 925 hPa WS MAX (m/s):  44.56
# 2, Latitude (deg):  -12.5, Longitude (deg):  167.5, MSLP (hPa):  982.3, 850 RV MAX (*10^-5 1/s):  4.05, 250-850 hPa Thickness (m): 9664.80, 925 hPa WS MAX (m/s):  44.56
# 3, Latitude (deg):   15.5, Longitude (deg):   53.5, MSLP (hPa):  992.7, 850 RV MAX (*10^-5 1/s): 49.59, 250-850 hPa Thickness (m): 9563.80, 925 hPa WS MAX (m/s):  57.83
# 4, Latitude (deg):   16.0, Longitude (deg):   84.5, MSLP (hPa):  986.1, 850 RV MAX (*10^-5 1/s): 55.36, 250-850 hPa Thickness (m): 9620.60, 925 hPa WS MAX (m/s):  58.66
# 5, Latitude (deg):   33.5, Longitude (deg):  -54.5, MSLP (hPa):  970.1, 850 RV MAX (*10^-5 1/s): 76.10, 250-850 hPa Thickness (m): 9682.10, 925 hPa WS MAX (m/s):  67.37
# 6, Latitude (deg):   38.0, Longitude (deg):   77.5, MSLP (hPa): 1016.5, 850 RV MAX (*10^-5 1/s):  2.59, 250-850 hPa Thickness (m): 9104.20, 925

In [11]:
#this example is 96h of NAVGEM
# output below for NAV IO..  (the # refers to our candidate #s)

# For MAX RV (in units *10^-5 1/s), FSU gets:
#   for #3: 49.5  versus our 49.59
#   for #4: 55.26 versus our 55.36
#   an error of about ~0.1%

# For 250-850 thickness (in units m), FSU gets:
#   for #3: 9563.8 versus our 9563.80
#   for #4: 9620.6 versus our 9620.60
#   an error of 0%

# For 925 WS MAX (in units m/s), FSU gets:
#   for #3:  57.83 versus our 57.83
#   for #4:  58.66  versus our 58.66
#   an error of 0%


#
# may have to do with differences in how we calculate relative vorticity ((finite differences) vorticity method from metpy),
#   or the way we calculate deltas for the dx/dy for the vorticity measurements (deltas method from metpy)

# NAVGEM from https://nrlgodae1.nrlmry.navy.mil/ftp/outgoing/fnmoc/models/navgem_0.5/2023/2023101906/US058GMET-GR1mdl.0018_0056_09600F0OF2023101906_0100_008500-000000rltv_vort
# does have it separately
# it has MAX RV for 850  48.1 at the lat lon 15.5, 53.5)
# +- 2 degrees is 13.5 : 17.5, and 51.5 to 55.5
# 49.3 is at (16.5, 54.5)
# 50.2 is at (16.5, 54.0)
# 52.1 is at (15.5, 54.0)
# 52.5 is at (15.5, 54.5)
# so max RV is 52.5

In [12]:
# from https://moe.met.fsu.edu/modelgen/text/nav/distinfo/distinfo_2023101906_NAV_IO.php
# #2 below is Invest 91A (#3 above), and #2 below is a disturbance (#4 above) in Bengal Bay 

In [13]:
# from https://moe.met.fsu.edu/modelgen/text/nav/distinfo/distinfo_2023101906_NAV_NATL.php
# #1 below is TS Tammy (#5 above)

# 5, Latitude (deg):   33.5, Longitude (deg):  -54.5, MSLP (hPa):  970.1, 850 RV MAX (*10^-5 1/s): 76.10, 250-850 hPa Thickness (m): 9682.10

# For MAX RV (in units *10^-5 1/s), FSU gets:
#   for #5: 76.1  versus our 76.10
#   an error of 0%

# For 250-850 thickness (in units m), FSU gets:
#   for #5: 9682.1 versus our 9682.1
#   an error of 0%

# For 925 WS MAX (in units m/s), FSU gets:
#   for #5:  67.37 versus our 67.37
#   an error of 0%


In [57]:
# Function to calculate the booleans on disturbance thresholds are met or not for each criteria and for all
# Must calculate and update with the basin classification first
def calc_disturbance_threshold_booleans(mslp_minima_list, model_name):
    updated_mslp_minima_list = []

    for candidate in mslp_minima_list:
        # Update a copy of the candidate's dictionary with rv maximum value
        updated_candidate = candidate.copy()
        all_met = True
        for criteria_abbrev, criteria_value in disturbance_thresholds[model_name]['IO'].items():
            criteria_name = disturbance_criteria_names_map[criteria_abbrev]
            if criteria_name == 'rv850max':
                # thresholds in the JSON for RV are scaled by 10^5
                criteria_value *= pow(10, -5)
            criteria_bool = False
            if updated_candidate[criteria_name] >= criteria_value:
                criteria_bool = True
            if 'criteria' not in updated_candidate:
                updated_candidate['criteria'] = {}
            updated_candidate['criteria'][criteria_name] = criteria_bool
            all_met = (all_met and criteria_bool)

        updated_candidate['criteria']['all'] = all_met
        # Add the updated candidate to the list
        updated_mslp_minima_list.append(updated_candidate)

    return updated_mslp_minima_list

In [64]:
mslp_minima_list_with_disturbance_threshold_booleans = calc_disturbance_threshold_booleans(mslp_minima_list_with_closed_isobars, model_name)
print_candidates(mslp_minima_list_with_disturbance_threshold_booleans, mslp_lats, mslp_lons, meet_all_disturbance_thresholds = True)

# 1, Latitude (deg):   15.5, Longitude (deg):   53.5, MSLP (hPa):  992.7, 850 RV MAX (*10^-5 1/s): 49.59, 250-850 hPa Thickness (m): 9563.80, 925 hPa WS MAX (m/s):  57.83
# 2, Latitude (deg):   16.0, Longitude (deg):   84.5, MSLP (hPa):  986.1, 850 RV MAX (*10^-5 1/s): 55.36, 250-850 hPa Thickness (m): 9620.60, 925 hPa WS MAX (m/s):  58.66
# 3, Latitude (deg):   33.5, Longitude (deg):  -54.5, MSLP (hPa):  970.1, 850 RV MAX (*10^-5 1/s): 76.10, 250-850 hPa Thickness (m): 9682.10, 925 hPa WS MAX (m/s):  67.37
